<h1>Question 5</h1>
<br>Duplicate Detection/Minhash in cookbook_txt.<br>
<b> NOTE: All the code is executed in databricks due to memory issues in jupyter.

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
import re

sc = SparkContext()
spark = SparkSession.builder.getOrCreate()

In [2]:
#convorting the regular expressions
def get_exp(pairs):
    line = re.sub(r'\n\s*\n','\n',pairs[1],re.MULTILINE)
    return [[[exp for exp in pairs[0].split('/')][-1] ,line]]

In [3]:
#Making an rdd by applying functions to all files in the folder
data_docs = sc.wholeTextFiles("/FileStore/tables/*.txt").flatMap(get_exp)

In [4]:
#Convorting the rdd to a dataframe for further processing
docs_info = data_docs.toDF(['doc_name','recipe'])
docs_info.show()

+--------+--------------------+
doc_name| recipe|
+--------+--------------------+
amem.txt| 
 The American M...|
amwh.txt| 
 The American...|
army.txt| 
 Manual For Ar...|
aunt.txt| 
 "Aunt Babett...|
bart.txt| 
 THE 
IDEAL B...|
beec.txt| 
 A bookplat...|
blue.txt| 
 THE BLUE GRAS...|
bost.txt| 
 THE BOSTON C...|
brkf.txt| 
 Breakfast, Lu...|
buck.txt| 
 Practical Ho...|
cclu.txt| 
 Cooking in ...|
chas.txt| 
 Dr. Chase's R...|
chin.txt| 
 Chinese-Japan...|
choc.txt| 
 Chocolate and...|
comm.txt| 
 Common Sense ...|
conf.txt| 
 The Complete ...|
coow.txt| 
 The Cook's Ow...|
creo.txt| 
 La Cuisine Cr...|
dcvb.txt| 
 Directions fo...|
dish.txt| 
 Dishes &amp;...|
+--------+--------------------+
only showing top 20 rows

In [5]:
#creating tokens and showing the tokenized data along with the prev data This gives us a set of values we can compare.
tokens = Tokenizer(inputCol="recipe", outputCol="tokens")
tokenized_data = tokens.transform(docs_info)
tokenized_data.show()

+--------+--------------------+--------------------+
doc_name| recipe| tokens|
+--------+--------------------+--------------------+
amem.txt| 
 The American M...|[, , , the, ameri...|
amwh.txt| 
 The American...|[, , , , , the, a...|
army.txt| 
 Manual For Ar...|[, , , , manual, ...|
aunt.txt| 
 "Aunt Babett...|[, , , , , "aunt,...|
bart.txt| 
 THE 
IDEAL B...|[, , , , the, , ,...|
beec.txt| 
 A bookplat...|[, , , , , , , a,...|
blue.txt| 
 THE BLUE GRAS...|[, , , , the, blu...|
bost.txt| 
 THE BOSTON C...|[, , , , the, , b...|
brkf.txt| 
 Breakfast, Lu...|[, , , , breakfas...|
buck.txt| 
 Practical Ho...|[, , , , practica...|
cclu.txt| 
 Cooking in ...|[, , , , , , cook...|
chas.txt| 
 Dr. Chase's R...|[, , , , dr., cha...|
chin.txt| 
 Chinese-Japan...|[, , , , chinese-...|
choc.txt| 
 Chocolate and...|[, , , , chocolat...|
comm.txt| 
 Common Sense ...|[, , , , common, ...|
conf.txt| 
 The Complete ...|[, , , , the, com...|
coow.txt| 
 The Cook's Ow...|[, , , , the, coo...|
creo.txt| 
 La Cuisine Cr...|[, , , , la, cuis...|
dcvb.txt| 
 Directions fo...|[, , , , directio...|
dish.txt| 
 Dishes &amp;...|[, , , , , dishes...|
+--------+--------------------+--------------------+
only showing top 20 rows

In [6]:
#Making shingles using NGram
n_grams = NGram(n=2, inputCol="tokens", outputCol="shingles")
shingled_data = n_grams.transform(tokenized_data)
shingled_data.show()

+--------+--------------------+--------------------+--------------------+
doc_name| recipe| tokens| shingles|
+--------+--------------------+--------------------+--------------------+
amem.txt| 
 The American M...|[, , , the, ameri...|[ , , the, the ...|
amwh.txt| 
 The American...|[, , , , , the, a...|[ , , , , the...|
army.txt| 
 Manual For Ar...|[, , , , manual, ...|[ , , , manual...|
aunt.txt| 
 "Aunt Babett...|[, , , , , "aunt,...|[ , , , , "au...|
bart.txt| 
 THE 
IDEAL B...|[, , , , the, , ,...|[ , , , the, t...|
beec.txt| 
 A bookplat...|[, , , , , , , a,...|[ , , , , , ...|
blue.txt| 
 THE BLUE GRAS...|[, , , , the, blu...|[ , , , the, t...|
bost.txt| 
 THE BOSTON C...|[, , , , the, , b...|[ , , , the, t...|
brkf.txt| 
 Breakfast, Lu...|[, , , , breakfas...|[ , , , breakf...|
buck.txt| 
 Practical Ho...|[, , , , practica...|[ , , , practi...|
cclu.txt| 
 Cooking in ...|[, , , , , , cook...|[ , , , , , ...|
chas.txt| 
 Dr. Chase's R...|[, , , , dr., cha...|[ , , , dr., d...|
chin.txt| 
 Chinese-Japan...|[, , , , chinese-...|[ , , , chines...|
choc.txt| 
 Chocolate and...|[, , , , chocolat...|[ , , , chocol...|
comm.txt| 
 Common Sense ...|[, , , , common, ...|[ , , , common...|
conf.txt| 
 The Complete ...|[, , , , the, com...|[ , , , the, t...|
coow.txt| 
 The Cook's Ow...|[, , , , the, coo...|[ , , , the, t...|
creo.txt| 
 La Cuisine Cr...|[, , , , la, cuis...|[ , , , la, la...|
dcvb.txt| 
 Directions fo...|[, , , , directio...|[ , , , direct...|
dish.txt| 
 Dishes &amp;...|[, , , , , dishes...|[ , , , , dis...|
+--------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [7]:
#Vectorizing the shingles
shingles_vec = CountVectorizer(inputCol="shingles", outputCol="shingle_vectors", vocabSize=10000, minDF=2.0)
model = shingles_vec.fit(shingled_data)
shingled_data = model.transform(shingled_data)
shingled_data.show()

+--------+--------------------+--------------------+--------------------+--------------------+
doc_name| recipe| tokens| shingles| shingle_vectors|
+--------+--------------------+--------------------+--------------------+--------------------+
amem.txt| 
 The American M...|[, , , the, ameri...|[ , , the, the ...|(10000,[0,4,21,21...|
amwh.txt| 
 The American...|[, , , , , the, a...|[ , , , , the...|(10000,[0,1,2,3,4...|
army.txt| 
 Manual For Ar...|[, , , , manual, ...|[ , , , manual...|(10000,[0,1,2,3,4...|
aunt.txt| 
 "Aunt Babett...|[, , , , , "aunt,...|[ , , , , "au...|(10000,[0,1,2,3,4...|
bart.txt| 
 THE 
IDEAL B...|[, , , , the, , ,...|[ , , , the, t...|(10000,[0,1,2,3,4...|
beec.txt| 
 A bookplat...|[, , , , , , , a,...|[ , , , , , ...|(10000,[0,1,2,3,4...|
blue.txt| 
 THE BLUE GRAS...|[, , , , the, blu...|[ , , , the, t...|(10000,[0,1,2,3,4...|
bost.txt| 
 THE BOSTON C...|[, , , , the, , b...|[ , , , the, t...|(10000,[0,1,2,3,4...|
brkf.txt| 
 Breakfast, Lu...|[, , , , breakfas...|[ , , , breakf...|(10000,[0,1,2,3,4...|
buck.txt| 
 Practical Ho...|[, , , , practica...|[ , , , practi...|(10000,[0,1,2,3,4...|
cclu.txt| 
 Cooking in ...|[, , , , , , cook...|[ , , , , , ...|(10000,[0,1,2,3,4...|
chas.txt| 
 Dr. Chase's R...|[, , , , dr., cha...|[ , , , dr., d...|(10000,[0,1,2,3,4...|
chin.txt| 
 Chinese-Japan...|[, , , , chinese-...|[ , , , chines...|(10000,[0,1,2,3,4...|
choc.txt| 
 Chocolate and...|[, , , , chocolat...|[ , , , chocol...|(10000,[0,1,2,3,4...|
comm.txt| 
 Common Sense ...|[, , , , common, ...|[ , , , common...|(10000,[0,1,2,3,4...|
conf.txt| 
 The Complete ...|[, , , , the, com...|[ , , , the, t...|(10000,[0,1,2,3,4...|
coow.txt| 
 The Cook's Ow...|[, , , , the, coo...|[ , , , the, t...|(10000,[0,1,2,3,4...|
creo.txt| 
 La Cuisine Cr...|[, , , , la, cuis...|[ , , , la, la...|(10000,[0,1,2,3,4...|
dcvb.txt| 
 Directions fo...|[, , , , directio...|[ , , , direct...|(10000,[0,1,2,3,4...|
dish.txt| 
 Dishes &amp;...|[, , , , , dishes...|[ , , , , dis...|(10000,[0,1,2,3,4...|
+--------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [8]:
#generating hash signatures using the min_hash algorithm 
min_hash = MinHashLSH(inputCol="shingle_vectors", outputCol="hashes", seed=12345, numHashTables=3)
model = min_hash.fit(shingled_data)
hashed_data = model.transform(shingled_data)
hashed_data.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
doc_name| recipe| tokens| shingles| shingle_vectors| hashes|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
amem.txt| 
 The American M...|[, , , the, ameri...|[ , , the, the ...|(10000,[0,4,21,21...|[[5.9439856E7], [...|
amwh.txt| 
 The American...|[, , , , , the, a...|[ , , , , the...|(10000,[0,1,2,3,4...|[[784957.0], [157...|
army.txt| 
 Manual For Ar...|[, , , , manual, ...|[ , , , manual...|(10000,[0,1,2,3,4...|[[482803.0], [659...|
aunt.txt| 
 "Aunt Babett...|[, , , , , "aunt,...|[ , , , , "au...|(10000,[0,1,2,3,4...|[[106485.0], [157...|
bart.txt| 
 THE 
IDEAL B...|[, , , , the, , ,...|[ , , , the, t...|(10000,[0,1,2,3,4...|[[973116.0], [375...|
beec.txt| 
 A bookplat...|[, , , , , , , a,...|[ , , , , , ...|(10000,[0,1,2,3,4...|[[106485.0], [157...|
blue.txt| 
 THE BLUE GRAS...|[, , , , the, blu...|[ , , , the, t...|(10000,[0,1,2,3,4...|[[294644.0], [157...|
bost.txt| 
 THE BOSTON C...|[, , , , the, , b...|[ , , , the, t...|(10000,[0,1,2,3,4...|[[106485.0], [157...|
brkf.txt| 
 Breakfast, Lu...|[, , , , breakfas...|[ , , , breakf...|(10000,[0,1,2,3,4...|[[106485.0], [157...|
buck.txt| 
 Practical Ho...|[, , , , practica...|[ , , , practi...|(10000,[0,1,2,3,4...|[[482803.0], [157...|
cclu.txt| 
 Cooking in ...|[, , , , , , cook...|[ , , , , , ...|(10000,[0,1,2,3,4...|[[482803.0], [659...|
chas.txt| 
 Dr. Chase's R...|[, , , , dr., cha...|[ , , , dr., d...|(10000,[0,1,2,3,4...|[[482803.0], [157...|
chin.txt| 
 Chinese-Japan...|[, , , , chinese-...|[ , , , chines...|(10000,[0,1,2,3,4...|[[106485.0], [120...|
choc.txt| 
 Chocolate and...|[, , , , chocolat...|[ , , , chocol...|(10000,[0,1,2,3,4...|[[482803.0], [157...|
comm.txt| 
 Common Sense ...|[, , , , common, ...|[ , , , common...|(10000,[0,1,2,3,4...|[[106485.0], [157...|
conf.txt| 
 The Complete ...|[, , , , the, com...|[ , , , the, t...|(10000,[0,1,2,3,4...|[[482803.0], [659...|
coow.txt| 
 The Cook's Ow...|[, , , , the, coo...|[ , , , the, t...|(10000,[0,1,2,3,4...|[[106485.0], [157...|
creo.txt| 
 La Cuisine Cr...|[, , , , la, cuis...|[ , , , la, la...|(10000,[0,1,2,3,4...|[[106485.0], [430...|
dcvb.txt| 
 Directions fo...|[, , , , directio...|[ , , , direct...|(10000,[0,1,2,3,4...|[[294644.0], [157...|
dish.txt| 
 Dishes &amp;...|[, , , , , dishes...|[ , , , , dis...|(10000,[0,1,2,3,4...|[[294644.0], [659...|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [9]:
# using approx similarity join for self joining and finding the distances among the document data
distances_val = model.approxSimilarityJoin(hashed_data, hashed_data, 3.0, distCol="eucDistance").select(col("datasetA.doc_name").alias("docA"),col("datasetB.doc_name").alias("docB"),col("eucDistance")).sort(desc("eucDistance")).dropDuplicates(['eucDistance'])

In [10]:
#Displaying the top 5 rows due to memory issues
distances_val.show(10)

+--------+--------+------------------+
 docA| docB| eucDistance|
+--------+--------+------------------+
notm.txt|bart.txt|0.8456949691787632|
time.txt|bart.txt|0.8545218145044785|
amwh.txt|fish.txt|0.7508479802651866|
zuni.txt|fran.txt|0.7659010600706714|
cclu.txt|youn.txt|0.7332290852228303|
army.txt|mary.txt|0.7342608206857786|
buck.txt|mary.txt|0.7342681654884691|
choc.txt|fran.txt| 0.726149752945648|
amwh.txt|fran.txt|0.7296062992125985|
chin.txt|choc.txt|0.7083247262962198|
+--------+--------+------------------+
only showing top 10 rows

In [11]:
#setting a threshold of 0.7 and filtering out the values over the threshold.
#Displaying the top 5 rows due to memory issues
threshold = 0.7
duplicates = distances_val.filter(distances_val['eucDistance'] > threshold)
duplicates.show(5)

+--------+--------+------------------+
 docA| docB| eucDistance|
+--------+--------+------------------+
notm.txt|bart.txt|0.8456949691787632|
time.txt|bart.txt|0.8545218145044785|
amwh.txt|fish.txt|0.7508479802651866|
zuni.txt|fran.txt|0.7659010600706714|
cclu.txt|youn.txt|0.7332290852228303|
+--------+--------+------------------+
only showing top 5 rows